In [23]:
import pandas as pd

trips = pd.read_csv('most_popular_trips.csv', 
                    dtype={'origin_state_county': tuple, 
                          'destination_state_county': tuple})
print(trips.shape)
trips.head()

TypeError: dtype '<class 'tuple'>' not understood

In [24]:
type((3, 4))

tuple

In [5]:
# Are there any counties of same name but different state?
trips['destination'].nunique()

1880

In [14]:
trips['destination_code'].nunique()

3145

In [8]:
pd.Series([str(tup) for tup in trips['destination_state_county']]).nunique()

3145

In [9]:
trips['destination_state_county'].value_counts()

('17', '031')    1600
('6', '037')     1561
('4', '013')     1420
('26', '163')    1312
('36', '063')    1302
                 ... 
('15', '009')       3
('15', '003')       2
('15', '005')       2
('2', '282')        2
('2', '013')        1
Name: destination_state_county, Length: 3145, dtype: int64

In [11]:
trips['destination'].value_counts()

Jefferson County            2963
Washington County           2507
Montgomery County           2388
Wayne County                2169
Franklin County             2007
                            ... 
Bristol Bay Borough            3
Honolulu County                2
Yakutat City and Borough       2
Kalawao County                 2
Aleutians East Borough         1
Name: destination, Length: 1880, dtype: int64

In [12]:
trips[trips['destination'] == 'Jefferson County']

,Unnamed: 0,origin_code,destination_code,n_trips,origin_state_county,destination_state_county,origin,destination
53,9356030,53033,53031,726104.0,"('53', '033')","('53', '031')",King County,Jefferson County
89,9387490,53053,53031,518652.0,"('53', '053')","('53', '031')",Pierce County,Jefferson County
185,9418950,53073,53031,292390.0,"('53', '073')","('53', '031')",Whatcom County,Jefferson County
668,9393782,53057,53031,139788.0,"('53', '057')","('53', '031')",Skagit County,Jefferson County
713,50372,1033,1073,131344.0,"('1', '033')","('1', '073')",Colbert County,Jefferson County
...,...,...,...,...,...,...,...,...
249230,4738715,29047,19101,1183.0,"('29', '047')","('19', '101')",Clay County,Jefferson County
249354,196100,1125,21111,1182.0,"('1', '125')","('21', '111')",Tuscaloosa County,Jefferson County
249471,7405720,45075,1073,1181.0,"('45', '075')","('1', '073')",Orangeburg County,Jefferson County
249657,6731459,40015,40067,1180.0,"('40', '015')","('40', '067')",Caddo County,Jefferson County


In [20]:
state_dest_codes = [pair for pair in trips['destination_state_county']]

In [21]:
state_dest_codes

["('6', '037')",
 "('6', '073')",
 "('6', '073')",
 "('6', '025')",
 "('6', '037')",
 "('6', '065')",
 "('6', '037')",
 "('6', '071')",
 "('53', '033')",
 "('53', '073')",
 "('4', '013')",
 "('4', '023')",
 "('6', '073')",
 "('6', '071')",
 "('6', '025')",
 "('6', '037')",
 "('6', '059')",
 "('6', '071')",
 "('6', '029')",
 "('6', '037')",
 "('4', '013')",
 "('4', '019')",
 "('6', '065')",
 "('6', '025')",
 "('32', '003')",
 "('4', '015')",
 "('4', '013')",
 "('4', '025')",
 "('6', '037')",
 "('6', '083')",
 "('6', '059')",
 "('6', '025')",
 "('4', '027')",
 "('6', '073')",
 "('6', '073')",
 "('6', '111')",
 "('6', '071')",
 "('6', '065')",
 "('4', '027')",
 "('4', '013')",
 "('4', '007')",
 "('4', '013')",
 "('6', '071')",
 "('6', '025')",
 "('48', '141')",
 "('35', '029')",
 "('6', '025')",
 "('4', '013')",
 "('4', '013')",
 "('4', '005')",
 "('4', '003')",
 "('4', '013')",
 "('53', '033')",
 "('53', '031')",
 "('6', '037')",
 "('6', '107')",
 "('48', '029')",
 "('48', '323')",
 "('3